In [1]:
import os

In [2]:
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/pyCharmProjects/Chicken-Disease-Classification--Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/pyCharmProjects/Chicken-Disease-Classification--Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.utils import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(config.checkpoint_model_filepath)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callbacks_config

In [22]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy as CC
from tensorflow.keras.callbacks import TensorBoard as TB
from tensorflow.keras.callbacks import ModelCheckpoint as MC
import time

In [23]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%s")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )

        return TB(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return MC(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [24]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e


[2023-12-16 00:05:35,051: INFO: utils: yaml file: config/config.yaml loaded successfully]
[2023-12-16 00:05:35,054: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-12-16 00:05:35,055: INFO: utils: created directory at: artifacts]
[2023-12-16 00:05:35,056: INFO: utils: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2023-12-16 00:05:35,057: INFO: utils: created directory at: artifacts/prepare_callbacks/checkpoint_dir/model.h5]
